# 导入包

In [1]:
#!/usr/bin/env python
# coding: utf-8
import pandas as pd
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import os
from time import time
import shutil
import argparse
import configparser
import copy
import os
import numpy as np
import torch
import torch.utils.data
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import numpy as np
from time import time
from scipy.sparse.linalg import eigs
import torch
import torch.nn as nn
import torch.nn.functional as F
import copy
import math
import numpy as np
from torch.nn.utils import weight_norm
import numpy as np
from torch.autograd import Variable
from torch.utils.data import TensorDataset, ConcatDataset, DataLoader
from torch.utils.data import DataLoader, TensorDataset, WeightedRandomSampler
from sklearn.utils import resample

In [2]:
def MinMaxnormalization(train, test ,_max ,_min):
    
    def normalize(x):
        x = 1. * (x - _min) / (_max - _min)
        x = 2. * x - 1.
        return x

    train_norm = normalize(train)
    test_norm = normalize(test)

    return train_norm,test_norm

In [3]:
def data_generator(root, param):
    dict_table = []
    # 读取CSV文件（假设文件名格式为"root_param.csv"，例如 "/data_A320_LDGL.csv"）
    X_train = pd.read_csv(root + "_" + param + ".csv")
    
    # 提取双标签：Label1 和 Label2
    Y_labels = ["Label1", "Label2"]
    Y = torch.from_numpy(X_train[Y_labels].values)  # 形状变为 [样本数, 2]
    
    # 构建特征数据（排除所有标签列）
    for v in X_train.drop(Y_labels, axis=1).values:
        table = []
        table.append(list(v))  # 将每个样本的特征序列包装为列表
        dict_table.append(table)
    
    # 转换为PyTorch张量：X形状为 [样本数, 1, 特征长度]
    X = torch.Tensor(dict_table)
    
    return X, Y

# 获取数据

In [4]:
# root = "/mnt/sdb/Dataset/LiCX/A320/origin/"
# X_1HZ, X_2HZ, X_4HZ, X_8HZ, Y = [], [], [], [], []
# select = 30
# #选择相应的参数
# param_list = [
#     'WIN_ALG','WIN_CRS','IAS','GS','VAPP','N11','N12','TLA1','TLA2','LDGR','LDGNOS','ALT_STD','RADIO_LH','RADIO_RH',
#     'IVV','VRTG','ROLL','ROLL_CMD','PITCH','PITCH_CMD','RUDD'
# ]
# param_list_1HZ, param_list_2HZ, param_list_4HZ, param_list_8HZ = [], [], [], []

# # 标志变量，用于确保Y只被提取一次
# labels_extracted = False

# for param in param_list:
#     # 加载CSV文件
#     df = pd.read_csv(root + param + ".csv")
    
#     # 提取双标签 (Label1和Label2)，只在第一次循环时提取
#     y_labels = ["Label1", "Label2"]
#     if not labels_extracted:  # 如果标签尚未提取
#         Y = torch.from_numpy(df[y_labels].values)  # 形状 [样本数, 2]
#         labels_extracted = True  # 标记为已提取
    
#     # 构建特征张量（排除Time和双标签列）
#     dict_table = []
#     for row in df.drop(["Time"] + y_labels, axis=1).values:
#         dict_table.append([list(row)])  # 保持三维结构 [样本数, 1, 时间步]
    
#     X = torch.Tensor(dict_table)
    
#     # 按采样率分类处理
#     time_steps = X.shape[2]
#     if time_steps == 50:       # 1Hz采样率
#         cut = select
#         X_1HZ.append(X[:, :, :cut])
#         param_list_1HZ.append(param)
#     elif time_steps == 100:    # 2Hz采样率
#         cut = select * 2
#         X_2HZ.append(X[:, :, :cut])
#         param_list_2HZ.append(param)
#     elif time_steps == 200:    # 4Hz采样率
#         cut = select * 4
#         X_4HZ.append(X[:, :, :cut])
#         param_list_4HZ.append(param)
#     else:                      # 8Hz采样率
#         cut = select * 8
#         X_8HZ.append(X[:, :, :cut])
#         param_list_8HZ.append(param)

# # 沿特征维度拼接（最终形状）
# X_1HZ_tensor = torch.cat(X_1HZ, dim=1)  # [样本数, 参数数_1HZ, 30]
# X_2HZ_tensor = torch.cat(X_2HZ, dim=1)  # [样本数, 参数数_2HZ, 60]
# X_4HZ_tensor = torch.cat(X_4HZ, dim=1)  # [样本数, 参数数_4HZ, 120]
# X_8HZ_tensor = torch.cat(X_8HZ, dim=1)  # [样本数, 参数数_8HZ, 240]

In [5]:
root = "/mnt/sdb/Dataset/LiCX/A320/origin/"
X_1HZ, X_2HZ, X_4HZ, X_8HZ, Y = [], [], [], [], []
select = 30
#选择相应的参数
param_list = [
    'WIN_ALG','WIN_CRS','IAS','GS','VAPP','N11','N12','TLA1','TLA2','LDGR','LDGNOS','ALT_STD','RADIO_LH','RADIO_RH',
    'IVV','VRTG','ROLL','ROLL_CMD','PITCH','PITCH_CMD','RUDD'
]
param_list_1HZ, param_list_2HZ, param_list_4HZ, param_list_8HZ = [], [], [], []

# 标志变量，用于确保Y只被提取一次
labels_extracted = False
for param in param_list:
    # 加载CSV文件
    df = pd.read_csv(root + param + ".csv")
    
    # 提取双标签 (Label1和Label2)，只在第一次循环时提取
    y_labels = ["Label1", "Label2"]
    if not labels_extracted:  # 如果标签尚未提取
        Y = torch.from_numpy(df[y_labels].values)  # 形状 [样本数, 2]
        labels_extracted = True  # 标记为已提取
    
    # 构建特征张量（排除Time和双标签列）
    dict_table = []
    for row in df.drop(["Time"] + y_labels, axis=1).values:
        dict_table.append([list(row)])  # 保持三维结构 [样本数, 1, 时间步]
    
    X = torch.Tensor(dict_table)
    
    # 按采样率分类处理
    time_steps = X.shape[2]
    print(f"参数: {param}, 时间步长: {time_steps}")  # 打印调试信息
    
    if time_steps == 100:       # 1Hz采样率
        cut = select
        X_1HZ.append(X[:, :, :cut])
        param_list_1HZ.append(param)
    elif time_steps == 200:    # 2Hz采样率
        cut = select * 2
        X_2HZ.append(X[:, :, :cut])
        param_list_2HZ.append(param)
    elif time_steps == 400:    # 4Hz采样率
        cut = select * 4
        X_4HZ.append(X[:, :, :cut])
        param_list_4HZ.append(param)
    elif time_steps == 800:    # 8Hz采样率
        cut = select * 8
        X_8HZ.append(X[:, :, :cut])
        param_list_8HZ.append(param)
    else:
        print(f"参数 {param} 的时间步长 {time_steps} 不符合预期采样率！")  # 打印警告信息
# 沿特征维度拼接（最终形状）
X_1HZ_tensor = torch.cat(X_1HZ, dim=1)  # [样本数, 参数数_1HZ, 30]
X_2HZ_tensor = torch.cat(X_2HZ, dim=1)  # [样本数, 参数数_2HZ, 60]
X_4HZ_tensor = torch.cat(X_4HZ, dim=1)  # [样本数, 参数数_4HZ, 120]
X_8HZ_tensor = torch.cat(X_8HZ, dim=1)  # [样本数, 参数数_8HZ, 240]

参数: WIN_ALG, 时间步长: 100
参数: WIN_CRS, 时间步长: 100
参数: IAS, 时间步长: 100
参数: GS, 时间步长: 100
参数: VAPP, 时间步长: 100
参数: N11, 时间步长: 100
参数: N12, 时间步长: 100
参数: TLA1, 时间步长: 100
参数: TLA2, 时间步长: 100
参数: LDGR, 时间步长: 400
参数: LDGNOS, 时间步长: 400
参数: ALT_STD, 时间步长: 100
参数: RADIO_LH, 时间步长: 400
参数: RADIO_RH, 时间步长: 400
参数: IVV, 时间步长: 100
参数: VRTG, 时间步长: 800
参数: ROLL, 时间步长: 200
参数: ROLL_CMD, 时间步长: 800
参数: PITCH, 时间步长: 400
参数: PITCH_CMD, 时间步长: 800
参数: RUDD, 时间步长: 200


In [6]:
param_list_8HZ

['VRTG', 'ROLL_CMD', 'PITCH_CMD']

# 划分数据集

In [7]:
split_line = int(len(Y) * 0.7)

In [8]:
train_X_1HZ_tensor=X_1HZ_tensor[:split_line]
train_X_2HZ_tensor=X_2HZ_tensor[:split_line]
train_X_4HZ_tensor=X_4HZ_tensor[:split_line]
train_X_8HZ_tensor=X_8HZ_tensor[:split_line]
train_Y_tensor=Y[:split_line]

In [9]:
test_X_1HZ_tensor=X_1HZ_tensor[split_line:]
test_X_2HZ_tensor=X_2HZ_tensor[split_line:]
test_X_4HZ_tensor=X_4HZ_tensor[split_line:]
test_X_8HZ_tensor=X_8HZ_tensor[split_line:]
test_Y_tensor=Y[split_line:]

In [10]:
max_1HZ = torch.amax(train_X_1HZ_tensor, dim=(0, 2), keepdims=True)
min_1HZ = torch.amin(train_X_1HZ_tensor, dim=(0, 2), keepdims=True)
max_2HZ = torch.amax(train_X_2HZ_tensor, dim=(0, 2), keepdims=True)
min_2HZ = torch.amin(train_X_2HZ_tensor, dim=(0, 2), keepdims=True)
max_4HZ = torch.amax(train_X_4HZ_tensor, dim=(0, 2), keepdims=True)
min_4HZ = torch.amin(train_X_4HZ_tensor, dim=(0, 2), keepdims=True)
max_8HZ = torch.amax(train_X_8HZ_tensor, dim=(0, 2), keepdims=True)
min_8HZ = torch.amin(train_X_8HZ_tensor, dim=(0, 2), keepdims=True)

In [11]:
train_X_1HZ_tensor_norm,test_X_1HZ_tensor_norm = MinMaxnormalization(train_X_1HZ_tensor, test_X_1HZ_tensor,max_1HZ,min_1HZ)
train_X_2HZ_tensor_norm,test_X_2HZ_tensor_norm = MinMaxnormalization(train_X_2HZ_tensor, test_X_2HZ_tensor,max_2HZ,min_2HZ)
train_X_4HZ_tensor_norm,test_X_4HZ_tensor_norm = MinMaxnormalization(train_X_4HZ_tensor, test_X_4HZ_tensor,max_4HZ,min_4HZ)
train_X_8HZ_tensor_norm,test_X_8HZ_tensor_norm = MinMaxnormalization(train_X_8HZ_tensor, test_X_8HZ_tensor,max_8HZ,min_8HZ)

In [12]:
dim, dim2,dim4,dim8,dim_test = train_X_1HZ_tensor_norm.shape[0],train_X_2HZ_tensor_norm.shape[1],train_X_4HZ_tensor_norm.shape[1],train_X_8HZ_tensor_norm.shape[1],test_X_1HZ_tensor_norm.shape[0]

# 归一化

In [13]:
train_X_2HZ_tensor_norm,test_X_2HZ_tensor_norm=train_X_2HZ_tensor_norm.reshape(dim, dim2,select,2),  test_X_2HZ_tensor_norm.reshape(dim_test, dim2,select,2)
train_X_4HZ_tensor_norm,test_X_4HZ_tensor_norm=train_X_4HZ_tensor_norm.reshape(dim, dim4,select,4),  test_X_4HZ_tensor_norm.reshape(dim_test, dim4,select,4)
train_X_8HZ_tensor_norm,test_X_8HZ_tensor_norm=train_X_8HZ_tensor_norm.reshape(dim, dim8,select,8),  test_X_8HZ_tensor_norm.reshape(dim_test, dim8,select,8)

# 不归一化

In [14]:
train_X_2HZ_tensor_norm,test_X_2HZ_tensor_norm=train_X_2HZ_tensor.reshape(dim, dim2,select,2),  test_X_2HZ_tensor.reshape(dim_test, dim2,select,2)
train_X_4HZ_tensor_norm,test_X_4HZ_tensor_norm=train_X_4HZ_tensor.reshape(dim, dim4,select,4),  test_X_4HZ_tensor.reshape(dim_test, dim4,select,4)
train_X_8HZ_tensor_norm,test_X_8HZ_tensor_norm=train_X_8HZ_tensor.reshape(dim, dim8,select,8),  test_X_8HZ_tensor.reshape(dim_test, dim8,select,8)

In [15]:
train_dataset = torch.utils.data.TensorDataset(train_X_1HZ_tensor_norm,train_X_2HZ_tensor_norm, train_X_4HZ_tensor_norm,train_X_8HZ_tensor_norm,train_Y_tensor)
test_dataset = torch.utils.data.TensorDataset(test_X_1HZ_tensor_norm,test_X_2HZ_tensor_norm, test_X_4HZ_tensor_norm,test_X_8HZ_tensor_norm,test_Y_tensor)

In [16]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 训练集上采样

In [17]:
indices_0 = torch.where(train_Y_tensor == 0)[0]
indices_1 = torch.where(train_Y_tensor == 1)[0]
# indices_0 = torch.where(test_Y_tensor == 0)[0]
# indices_1 = torch.where(test_Y_tensor == 1)[0]

In [18]:
indices_1_upsampled = resample(indices_1, replace=True, n_samples=len(indices_0), random_state=42)

In [19]:
balanced_indices = torch.cat([indices_0, indices_1_upsampled])

In [20]:
train_X_1HZ_tensor_norm_balanced = train_X_1HZ_tensor_norm[balanced_indices]
train_X_2HZ_tensor_norm_balanced = train_X_2HZ_tensor_norm[balanced_indices]
train_X_4HZ_tensor_norm_balanced = train_X_4HZ_tensor_norm[balanced_indices]
train_X_8HZ_tensor_norm_balanced = train_X_8HZ_tensor_norm[balanced_indices]
train_Y_tensor_balanced = train_Y_tensor[balanced_indices]

In [21]:
shuffled_indices = torch.randperm(len(train_Y_tensor_balanced))
train_X_1HZ_tensor_norm_balanced = train_X_1HZ_tensor_norm_balanced[shuffled_indices]
train_X_2HZ_tensor_norm_balanced = train_X_2HZ_tensor_norm_balanced[shuffled_indices]
train_X_4HZ_tensor_norm_balanced = train_X_4HZ_tensor_norm_balanced[shuffled_indices]
train_X_8HZ_tensor_norm_balanced = train_X_8HZ_tensor_norm_balanced[shuffled_indices]
train_Y_tensor_balanced = train_Y_tensor_balanced[shuffled_indices]

In [22]:
train_dataset = torch.utils.data.TensorDataset(
    train_X_1HZ_tensor_norm_balanced, 
    train_X_2HZ_tensor_norm_balanced, 
    train_X_4HZ_tensor_norm_balanced, 
    train_X_8HZ_tensor_norm_balanced, 
    train_Y_tensor_balanced
)

In [23]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

## Block

In [24]:
class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()


class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.conv2, self.chomp2, self.dropout1)
        
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)


class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.5):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

In [25]:
class GRUCell(nn.Module):
    def __init__(self, input_HZ, hidden_size):
        super(GRUCell, self).__init__()
        self.hidden_size = hidden_size
        
        # 更新门的权重和偏置
        self.W_z = nn.Linear(input_HZ, hidden_size)
        self.U_z = nn.Linear(hidden_size, hidden_size)
        
        # 重置门的权重和偏置
        self.W_r = nn.Linear(input_HZ, hidden_size)
        self.U_r = nn.Linear(hidden_size, hidden_size)
        
        # 候选隐藏状态的权重和偏置
        self.W_h = nn.Linear(input_HZ, hidden_size)
        self.U_h = nn.Linear(hidden_size, hidden_size)

    def forward(self, x, h):
        # 更新门 z_t
        z_t = torch.sigmoid(self.W_z(x) + self.U_z(h))
        
        # 重置门 r_t
        r_t = torch.sigmoid(self.W_r(x) + self.U_r(h))
        
        # 候选隐藏状态 h_tilda
        h_tilda = torch.tanh(self.W_h(x) + self.U_h(r_t * h))
        
        # 当前隐藏状态 h_t
        h_t = (1 - z_t) * h + z_t * h_tilda
        
        return h_t
class GRU(nn.Module):
    def __init__(self, input_HZ, hidden_size):
        super(GRU, self).__init__()
        self.hidden_size = hidden_size
        self.gru_cell = nn.GRUCell(input_HZ, hidden_size)
        self.linear=nn.Linear(hidden_size,input_HZ)

    def forward(self, x, h0=None):
        """
        :param x: 输入序列，形状为 (batch_size, HZ, seq_len)
        :param h0: 初始隐藏状态，形状为 (batch_size, hidden_size)
        :return: 最后的隐藏状态和输出序列
        """
        batch_size, _, seq_len = x.size()
        
        if h0 is None:
            h0 = torch.zeros(batch_size, self.hidden_size, device=x.device)

        h_t = h0
        outputs = []

        for t in range(seq_len):
            x_t = x[:, :, t]
            h_t = self.gru_cell(x_t, h_t)
            outputs.append(h_t.unsqueeze(1))

        outputs = torch.cat(outputs, dim=1)
#         h_tt=self.linear(h_t)
        return outputs, h_t

## IMVTCN

In [26]:
class IMVTCN(nn.Module):
    def __init__(self, param_sizes, input_size, time_output_size, output_size, num_channels, kernel_size, dropout, num_1HZ, num_2HZ, num_4HZ, num_8HZ, time_forward=0, imbalance_data=False, isDiffKernel=False, gap=False, out_middle_layer=False, out_path=""):
        super(IMVTCN, self).__init__()
        parallel_net1, parallel_net2, parallel_net4, parallel_net8 = [], [], [], []
        parallel_linear1, parallel_linear2, parallel_linear4, parallel_linear8 = [], [], [], []
        self.minsize = np.min(param_sizes) - 1
        self.num_1HZ = num_1HZ
        self.num_2HZ = num_2HZ
        self.num_4HZ = num_4HZ
        self.num_8HZ = num_8HZ
        parallel_gru1, parallel_gru2, parallel_gru4, parallel_gru8 = [], [], [], []
        for i in range(num_1HZ):
            parallel_gru1 += [nn.Linear(param_sizes, 30)]
        for i in range(num_2HZ):
            parallel_gru2 += [GRU(param_sizes, 30)]
        for i in range(num_4HZ):
            parallel_gru4 += [GRU(param_sizes, 30)]
        for i in range(num_8HZ):
            parallel_gru8 += [GRU(param_sizes, 30)]

        for i in range(num_1HZ):
            parallel_net1 += [nn.Linear(1 * 100, 30)]
            parallel_linear1 += [nn.Linear(num_channels[-1], time_output_size)]
        for i in range(num_2HZ):
            parallel_net2 += [nn.Linear(2 * 100, 30)]
            parallel_linear2 += [nn.Linear(num_channels[-1], time_output_size)]
        for i in range(num_4HZ):
            parallel_net4 += [nn.Linear(4 * 100, 30)]
            parallel_linear4 += [nn.Linear(num_channels[-1], time_output_size)]
        for i in range(num_8HZ):
            parallel_net8 += [nn.Linear(8 * 100, 30)]
            parallel_linear8 += [nn.Linear(num_channels[-1], time_output_size)]
        self.dtimef = time_forward
        self.gap = gap
        self.im = imbalance_data
        self.oml = out_middle_layer
        if self.oml: self.out_path = out_path
        self.net1 = nn.ModuleList(parallel_gru1)
        self.net2 = nn.ModuleList(parallel_gru2)
        self.net4 = nn.ModuleList(parallel_gru4)
        self.net8 = nn.ModuleList(parallel_gru8)
        self.linear1 = nn.ModuleList(parallel_linear1)
        self.linear2 = nn.ModuleList(parallel_linear2)
        self.linear4 = nn.ModuleList(parallel_linear4)
        self.linear8 = nn.ModuleList(parallel_linear8)
        self.relu = nn.ReLU()
        self.linear_task1 = nn.Linear(time_output_size * (num_1HZ + num_2HZ + num_4HZ + num_8HZ), output_size)
        self.linear_task2 = nn.Linear(time_output_size * (num_1HZ + num_2HZ + num_4HZ + num_8HZ), output_size)

    def select_data(self, time, ys):
        y_list = []
        for i in range(len(ys)):
            y = ys[i]
            ti = int(time[i][0].item())
            t = ti - 1 if (ti != 0) and (ti % self.minsize == 0) else ti
            t = int(t - self.dtimef * (y.shape[1] / self.minsize))
            if self.im:
                if self.gap:
                    sl = torch.mean(y[:, :t + 1], 1)
                else:
                    sl = y[:, t]
            else:
                if self.gap:
                    sl = torch.mean(y[:, :t + 1], 1).cpu().detach().numpy().tolist()
                else:
                    sl = y[:, t].cpu().detach().numpy().tolist()
            y_list.append(sl)
        if self.im: return torch.stack(y_list, 0).view(len(y_list), -1)
        else: return Variable(torch.Tensor(y_list).cuda())

    def forward(self, X_1HZ, X_2HZ, X_4HZ, X_8HZ):
        """(batch_size, num_2HZ, 50, 2)"""
        out_list = []
        start_point = 0
        batch_size = X_1HZ.shape[0]
        for i, net in enumerate(self.net1):
            y1 = net(X_1HZ[:, i, :])
            o = self.linear1[i](y1)
            out_list.append(o)
        for i, net in enumerate(self.net2):
            y2 = net(X_2HZ[:, i, :, :])[1]
            self.linear2[i](y2)
            out_list.append(o)
        for i, net in enumerate(self.net4):
            y4 = net(X_4HZ[:, i, :, :])[1]
            o = self.linear4[i](y4)
            out_list.append(o)
        for i, net in enumerate(self.net8):
            y8 = net(X_8HZ[:, i, :, :])[1]
            o = self.linear8[i](y8)
            out_list.append(o)
        cat_feature = torch.cat((out_list), dim=1)
        out_task1 = self.linear_task1(cat_feature) #重着陆
        out_task2 = self.linear_task2(cat_feature) #擦机尾
        return [out_task1, out_task2]


## 设置模型参数

In [27]:
#设置param_sizes、namelist
param_sizes = [51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 
               201, 201, 201, 51, 51, 201, 201, 51, 101, 401, 201, 401, 51, 101]
namelist = ['WIN_ALG','WIN_CRS','IAS','GS','VAPP','N11','N12','TLA1','TLA2','DME1','DME2','FLAP_PL','FLAP_PR',
              'LDGL','LDGR','LDGNOS','ALT_QNH','ALT_STD','RADIO_LH','RADIO_RH',
              'IVV','ROLL','ROLL_CMD','PITCH','PITCH_CMD','GW','RUDD']
# base = "./data/A320/origin/"
# param_sizes = []
# for p in namelist:
#     df = pd.read_csv(base+p+".csv")
#     l = len(df.columns.tolist())-1
#     print(p,l)
#     param_sizes.append(l)

In [28]:
server = 75 #服务器ID，76时并行多块GPU运行
batch_size = 64
epochs = 100
input_channels = 1  #输入通道 1
time_output_size = 1 #实验点 动态调整 每个参数几个特征表示 1 2 3 4
n_classes = 2   # 输出 2分类问题
layers = 3 # 动态调整 层数 3 4 5 6
nhid = 30 #隐藏层个数
channel_sizes = [nhid]*layers
kernel_size = 5 # 动态调整 卷积核尺度 3 5 7 9
dropout = 0.5 # 根据需求调整 0.5 0.2 0.05
time_forward = 0 # 动态调整 选取时间点 取0时：VRTG峰值或完成接地时刻  0 2 4
imbalance_data = True
imrate = 133.55 # 样本的比例
isDiffKernel = False  # 动态调整 是否每个子模型卷积核不相同
gap = False # 动态调整 是否用全局平均池化
out_middle_layer = False #是否输出模型中间产生数据 要求time_forward = 0, time_output_size = 1时输出，最后一次迭代
out_path = "/mnt/sdb/Dataset/LiCX/"
model_save = "/mnt/sdb/Dataset/LiCX/" # 存储训练好的模型

sys.path.append("../../")
parser = argparse.ArgumentParser(description='Sequence Modeling - Safety incidents')
parser.add_argument('--cuda', action='store_false',
                    help='use CUDA (default: True)')
parser.add_argument('--clip', type=float, default=-1,
                    help='gradient clip, -1 means no clip (default: -1)')
parser.add_argument('--log-interval', type=int, default=25, metavar='N',
                    help='report interval (default: 100')
parser.add_argument('--seed', type=int, default=1111,
                    help='random seed (default: 1111)')
parser.add_argument('--lr', type=float, default=1e-2,
                    help='initial learning rate (default: 1e-2)')
parser.add_argument('--optim', type=str, default='Adam',
                    help='optimizer to use (default: Adam)')
args = parser.parse_known_args()[0]

torch.manual_seed(args.seed)
if torch.cuda.is_available():
    if not args.cuda:
        print("WARNING: You have a CUDA device, so you should probably run with --cuda")

# 多任务损失函数
if imbalance_data:
    loss_func_task1 = nn.CrossEntropyLoss(weight=torch.tensor([1, imrate], device='cuda'))  # 任务1的损失
    loss_func_task2 = nn.CrossEntropyLoss(weight=torch.tensor([1, imrate], device='cuda'))  # 任务2的损失
else:
    loss_func_task1 = nn.CrossEntropyLoss()  # 任务1的损失
    loss_func_task2 = nn.CrossEntropyLoss()  # 任务2的损失

In [29]:
model = IMVTCN(select,input_channels, time_output_size, 
                   n_classes, channel_sizes, kernel_size=kernel_size, dropout=dropout,num_1HZ=len(param_list_1HZ),num_2HZ=len(param_list_2HZ),num_4HZ=len(param_list_4HZ),num_8HZ=len(param_list_8HZ),
                  time_forward= time_forward, imbalance_data = imbalance_data, 
                   isDiffKernel = isDiffKernel, gap = gap,
                   out_middle_layer = out_middle_layer, out_path = out_path) 

## 训练测试

In [30]:
def mkdir(path):
    path=path.strip()
    path=path.rstrip("\\")
    path=path.rstrip("/")
    isExists=os.path.exists(path)
    if not isExists:
        os.makedirs(path) 


    
    
def train(train_loader, model, epoch):
    global lr, steps
    model.train()
    batch_idx = 1
    train_loss = 0
    steps = 0
    correct_task1, correct_task2 = 0, 0
    tp_task1, tn_task1, fn_task1, fp_task1 = 0, 0, 0, 0
    tp_task2, tn_task2, fn_task2, fp_task2 = 0, 0, 0, 0
    for batch_index, batch_data in enumerate(train_loader):
        batch_size = batch_data[0].shape[0]
        num_1HZ = batch_data[0].shape[1]
        num_2HZ = batch_data[1].shape[1]
        num_4HZ = batch_data[2].shape[1]
        num_8HZ = batch_data[3].shape[1]
        input_1HZ = batch_data[0].cuda()
        input_2HZ = batch_data[1].reshape(batch_size, num_2HZ, select, 2).cuda()
        input_4HZ = batch_data[2].reshape(batch_size, num_4HZ, select, 4).cuda()
        input_8HZ = batch_data[3].reshape(batch_size, num_8HZ, select, 8).cuda()
        y = batch_data[4].cuda()  # y 的形状应为 (batch_size, 2)
        input_1HZ, input_2HZ, input_4HZ, input_8HZ, y = Variable(input_1HZ), Variable(input_2HZ), Variable(input_4HZ), Variable(input_8HZ), Variable(y)
        seq_length = batch_size
        optimizer.zero_grad()
        device = torch.cuda.current_device()
        output = model(input_1HZ, input_2HZ, input_4HZ, input_8HZ)
        out_task1, out_task2 = output  # 获取两个任务的输出

        # 计算损失
        loss_task1 = loss_func_task1(out_task1, y[:, 0])  # 任务1的损失
        loss_task2 = loss_func_task2(out_task2, y[:, 1])  # 任务2的损失
        loss = loss_task1 + loss_task2  # 总损失
        loss.backward()

        if args.clip > 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip)
        optimizer.step()

        training_loss = loss.item()
        batch_idx += 1
        train_loss += training_loss * batch_size
        steps += batch_size

        # 计算任务1的指标
        pred_task1 = out_task1.data.max(1, keepdim=True)[1]
        correct_task1 += pred_task1.eq(y[:, 0].data.view_as(pred_task1)).cpu().sum()
        tp_task1 += ((pred_task1.data.view_as(y[:, 0].data) == 1) & (y[:, 0].data == 1)).cpu().sum()
        tn_task1 += ((pred_task1.data.view_as(y[:, 0].data) == 0) & (y[:, 0].data == 0)).cpu().sum()
        fn_task1 += ((pred_task1.data.view_as(y[:, 0].data) == 0) & (y[:, 0].data == 1)).cpu().sum()
        fp_task1 += ((pred_task1.data.view_as(y[:, 0].data) == 1) & (y[:, 0].data == 0)).cpu().sum()

        # 计算任务2的指标
        pred_task2 = out_task2.data.max(1, keepdim=True)[1]
        correct_task2 += pred_task2.eq(y[:, 1].data.view_as(pred_task2)).cpu().sum()
        tp_task2 += ((pred_task2.data.view_as(y[:, 1].data) == 1) & (y[:, 1].data == 1)).cpu().sum()
        tn_task2 += ((pred_task2.data.view_as(y[:, 1].data) == 0) & (y[:, 1].data == 0)).cpu().sum()
        fn_task2 += ((pred_task2.data.view_as(y[:, 1].data) == 0) & (y[:, 1].data == 1)).cpu().sum()
        fp_task2 += ((pred_task2.data.view_as(y[:, 1].data) == 1) & (y[:, 1].data == 0)).cpu().sum()

    train_loss = train_loss / steps
    accuracy_task1 = correct_task1 / steps
    accuracy_task2 = correct_task2 / steps

    # 计算任务1的指标
    p_task1 = tp_task1.item() / (tp_task1.item() + fp_task1.item())
    r_task1 = tp_task1.item() / (tp_task1.item() + fn_task1.item())
    F1_task1 = 2 * r_task1 * p_task1 / (r_task1 + p_task1)

    # 计算任务2的指标
    p_task2 = tp_task2.item() / (tp_task2.item() + fp_task2.item())
    r_task2 = tp_task2.item() / (tp_task2.item() + fn_task2.item())
    F1_task2 = 2 * r_task2 * p_task2 / (r_task2 + p_task2)

    # 打印训练指标
    print(f'Epoch {epoch}, Train Loss: {train_loss:.4f}, '
          f'Task 1 - Accuracy: {accuracy_task1:.4f}, Precision: {p_task1:.4f}, Recall: {r_task1:.4f}, F1: {F1_task1:.4f}, '
          f'Task 2 - Accuracy: {accuracy_task2:.4f}, Precision: {p_task2:.4f}, Recall: {r_task2:.4f}, F1: {F1_task2:.4f}')
    
    
def test(test_loader, model, epoch):
    global lr, steps
    model.eval()  # 注意：测试时模型应处于 eval 模式
    batch_idx = 1
    test_loss = 0
    steps = 0
    correct_task1, correct_task2 = 0, 0
    tp_task1, tn_task1, fn_task1, fp_task1 = 0, 0, 0, 0
    tp_task2, tn_task2, fn_task2, fp_task2 = 0, 0, 0, 0
    with torch.no_grad():  # 测试时不需要计算梯度
        for batch_index, batch_data in enumerate(test_loader):
            batch_size = batch_data[0].shape[0]
            num_1HZ = batch_data[0].shape[1]
            num_2HZ = batch_data[1].shape[1]
            num_4HZ = batch_data[2].shape[1]
            num_8HZ = batch_data[3].shape[1]
            input_1HZ = batch_data[0].cuda()
            input_2HZ = batch_data[1].reshape(batch_size, num_2HZ, select, 2).cuda()
            input_4HZ = batch_data[2].reshape(batch_size, num_4HZ, select, 4).cuda()
            input_8HZ = batch_data[3].reshape(batch_size, num_8HZ, select, 8).cuda()
            y = batch_data[4].cuda()  # y 的形状应为 (batch_size, 2)
            input_1HZ, input_2HZ, input_4HZ, input_8HZ, y = Variable(input_1HZ), Variable(input_2HZ), Variable(input_4HZ), Variable(input_8HZ), Variable(y)
            seq_length = batch_size
            output = model(input_1HZ, input_2HZ, input_4HZ, input_8HZ)
            out_task1, out_task2 = output  # 获取两个任务的输出

            # 计算损失
            loss_task1 = loss_func_task1(out_task1, y[:, 0])  # 任务1的损失
            loss_task2 = loss_func_task2(out_task2, y[:, 1])  # 任务2的损失
            loss = loss_task1 + loss_task2  # 总损失

            test_loss += loss.item() * batch_size
            steps += batch_size

            # 计算任务1的指标
            pred_task1 = out_task1.data.max(1, keepdim=True)[1]
            correct_task1 += pred_task1.eq(y[:, 0].data.view_as(pred_task1)).cpu().sum()
            tp_task1 += ((pred_task1.data.view_as(y[:, 0].data) == 1) & (y[:, 0].data == 1)).cpu().sum()
            tn_task1 += ((pred_task1.data.view_as(y[:, 0].data) == 0) & (y[:, 0].data == 0)).cpu().sum()
            fn_task1 += ((pred_task1.data.view_as(y[:, 0].data) == 0) & (y[:, 0].data == 1)).cpu().sum()
            fp_task1 += ((pred_task1.data.view_as(y[:, 0].data) == 1) & (y[:, 0].data == 0)).cpu().sum()

            # 计算任务2的指标
            pred_task2 = out_task2.data.max(1, keepdim=True)[1]
            correct_task2 += pred_task2.eq(y[:, 1].data.view_as(pred_task2)).cpu().sum()
            tp_task2 += ((pred_task2.data.view_as(y[:, 1].data) == 1) & (y[:, 1].data == 1)).cpu().sum()
            tn_task2 += ((pred_task2.data.view_as(y[:, 1].data) == 0) & (y[:, 1].data == 0)).cpu().sum()
            fn_task2 += ((pred_task2.data.view_as(y[:, 1].data) == 0) & (y[:, 1].data == 1)).cpu().sum()
            fp_task2 += ((pred_task2.data.view_as(y[:, 1].data) == 1) & (y[:, 1].data == 0)).cpu().sum()

    test_loss = test_loss / steps
    accuracy_task1 = correct_task1 / steps
    accuracy_task2 = correct_task2 / steps

    # 计算任务1的指标
    p_task1 = tp_task1.item() / (tp_task1.item() + fp_task1.item())
    r_task1 = tp_task1.item() / (tp_task1.item() + fn_task1.item())
    F1_task1 = 2 * r_task1 * p_task1 / (r_task1 + p_task1)

    # 计算任务2的指标
    p_task2 = tp_task2.item() / (tp_task2.item() + fp_task2.item())
    r_task2 = tp_task2.item() / (tp_task2.item() + fn_task2.item())
    F1_task2 = 2 * r_task2 * p_task2 / (r_task2 + p_task2)

    # 打印测试指标
    print(f'Epoch {epoch}, Test Loss: {test_loss:.4f}, '
          f'Task 1 - Accuracy: {accuracy_task1:.4f}, Precision: {p_task1:.4f}, Recall: {r_task1:.4f}, F1: {F1_task1:.4f}, '
          f'Task 2 - Accuracy: {accuracy_task2:.4f}, Precision: {p_task2:.4f}, Recall: {r_task2:.4f}, F1: {F1_task2:.4f}')

In [31]:
if __name__ == "__main__":
#     savedStdout = sys.stdout  #保存标准输出流
#     f = open(log_out, 'a') 
#     sys.stdout = f  #输出流变成文件
    for m in range(1):
#         model = model_list[m]
        if str(server) == "76":
            model = nn.DataParallel(model,device_ids=[0,1])
        lr = args.lr
        optimizer = getattr(optim, args.optim)(model.parameters(), lr=lr)
        if args.cuda:
            model.cuda()
        acc_list_train,acc_list_test,pre_train,pre_test,re_train,re_test,f1_train,f1_test = [],[],[],[],[],[],[],[]
        loss_train,loss_test = [],[]
        for epoch in range(1, epochs+1):
            train(train_loader,model,epoch)
            test(test_loader,model,epoch)
            if epoch % 10 == 0:
                lr /= 10
                for param_group in optimizer.param_groups:
                    param_group['lr'] = lr
        # 保存整个网络
#         torch.save(model,model_save+"imvtcn_model_"+model_index[m]+".pt") 
#         # 保存网络中的参数, 速度快，占空间少
#         torch.save(model.state_dict(),model_save+"imvtcn_hyperparam_"+model_index[m]+".pt")
#         outdf = pd.DataFrame({"Acc_train":acc_list_train,"Acc_test":acc_list_test,"Pre_train":pre_train,"Pre_test":pre_test,
#                      "Recall_train":re_train,"Recall_test":re_test,"F1_train":f1_train,"F1_test":f1_test,"loss_train":loss_train,
#                              "loss_test":loss_test})
#         outdf.to_csv(model_save+"evaluate_imvtcn_"+model_index[m]+".csv",index = False)
        torch.cuda.empty_cache()
#     sys.stdout = savedStdout  #恢复标准输出流
#     print ('已输出结果，恢复标准输出流!')
#     f.close() 
        

In [ ]:
model

NameError: name 'model' is not defined